<a href="https://colab.research.google.com/github/henthornlab/ProcessAnalytics/blob/master/Sensorpush.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Sensorpush Access Notebook**#
# David B. Henthorn, Dept. of Chemical Engineering,
Rose-Hulman Institute of Technology

<img style="float: right;" src="https://raw.githubusercontent.com/henthornlab/ProcessAnalytics/master/RHITlogo.png">




In [1]:
import requests
import ast
from io import StringIO
import pandas as pd
from datetime import timedelta  
import plotly.express as px

Accessing data requires a token. But before we get the token, we need to first supply a username and password to be authorized. 

In [2]:
authHeaders = {'accept': 'application/json', 'Content-Type': 'application/json'}
authUrl = "https://api.sensorpush.com/api/v1/oauth/authorize"
authBody = '{"email": "x@y", "password": "zzzzz"}'

authResponse = requests.post(authUrl, headers=authHeaders, data = authBody)

if authResponse.status_code != 200:
    print('# Request failed with code:', authResponse.status_code, authResponse.text)
else:
    print('Authorized')

Authorized


After authorization, we now ask for a token.

In [3]:
tokenHeaders = {'accept': 'application/json', 'Content-Type': 'application/json'}
tokenUrl = "https://api.sensorpush.com/api/v1/oauth/accesstoken"
tokenBody = authResponse.text

tokenResponse = requests.post(tokenUrl, headers=tokenHeaders, data = tokenBody)

if tokenResponse.status_code != 200:
    print('# Request failed with code:', tokenResponse.status_code, tokenResponse.text)
else:
    token = ast.literal_eval(tokenResponse.text)['accesstoken']
    print("Success!")

Success!


We can ask for the last x number of data points using the /samples endpoint. The provided data is easier work with in CSV form, ask for that with format:csv 

In [4]:
queryHeaders = {'accept': 'application/json', 'Authorization': token}
queryUrl = "https://api.sensorpush.com/api/v1/samples"
queryBody = '{ "limit": 1000, "format":"csv" }'

#print(queryHeaders)

queryResponse = requests.post(queryUrl, headers=queryHeaders, data = queryBody)

if queryResponse.status_code != 200:
    print('# Request failed with code:', queryResponse.status_code, queryResponse.text)
else:
    print('Success!')

Success!


Let's take the string of data and format it into a Pandas dataframe. First pass it though StringIO to clean it up.  

Next, adjust the provided time (UTC) to match the local timezone.

In [5]:
    df = pd.read_csv(StringIO(queryResponse.text), skiprows=2)
    df['Time'] = pd.to_datetime(df['observed'], infer_datetime_format=True) + timedelta(hours = -5)

Time to plot temperature and then humidity.

In [6]:
fig = px.scatter(df, x = 'Time', y = 'temperature')
fig.update_layout(
    title="UOLAB Temperatures",
    xaxis_title='Time (Eastern)',
    yaxis_title="Temperature",
    font=dict(
        family="Arial",
        size=16
    )
)
fig.show()

In [7]:
fig2 = px.scatter(df, x = 'Time', y = 'humidity')
fig2.update_layout(
    title="UOLAB Relative Humidity",
    xaxis_title='Time (Eastern)',
    yaxis_title="Relative Humidity%",
    font=dict(
        family="Arial",
        size=16
    )
)
fig2.show()